In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from datetime import datetime
from scipy import stats


import requests
import json


In [3]:

from matplotlib import style
style.use('fivethirtyeight')

import datetime as dt

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from sqlalchemy.ext.declarative import declarative_base

import sqlite3

Loading Data

In [4]:
# Read saved data: Registers and Local DB
QLD_IRA_db = pd.read_csv("Resources/QLD_IRA.csv", encoding="utf-8")

# Display sample data
QLD_IRA_db.head()



,IRAApparatusID,ApparatusType,Manufacturer,Model,SerialNumber,TubeDetails,LocationStorage,LocationUse,LocationRelocatingto,SN
0,I-AT46800,14 MeV neutron generator incorporating a H3 t...,Schlumberger Technology Corporation,PNG-GC,PNG-GC 67123-50503,No data,85 Duke Street ROMA QLD 4455,NaN,NaN,67123-50503
1,X61962,Linear accelerator apparatus,Schlumberger Technology Corporation,PNG-CB,8294-50388,No data,85 Duke Street ROMA QLD 4455,Throughout Queensland,NaN,8294-50388
2,X61963,Linear accelerator apparatus,Schlumberger Technology Corporation,PNG-CB,82596-43320,No data,85 Duke Street ROMA QLD 4455,Throughout Queensland,NaN,82596-43320
3,X61964,Linear accelerator apparatus,Schlumberger Technology Corporation,PNG-CB,8735-43534,No data,85 Duke Street ROMA QLD 4455,Throughout Queensland,NaN,8735-43534
4,X61965,Linear accelerator apparatus,Schlumberger Technology Corporation,PNG-CB,8864-50705,No data,85 Duke Street ROMA QLD 4455,Throughout Queensland,NaN,8864-50705


In [5]:
# Read saved data: Registers and Local DB
QLD_Reg_AR_db = pd.read_csv("Resources/QLD_Reg_AR.csv", encoding="utf-8")

# Display sample data
QLD_Reg_AR_db.head()



,ApparatusID,ApparatusManufacturer,Model,ApparatusSerialNumber,Purpose,SubstanceID,Radionuclide,Activity,MeasurementDate,UseLocation,StorageLocation,BusinessLine,ATA,Certificate,COC,InRMIS-NG,Status,FeedbackTIM,StatusShort,Evidence
0,I-AT46800,NaN,PNG-GC,67123-50503,NaN,NaN,NaN,NaN,NaN,NaN,85 Duke Street,RPE,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN
1,X61962,NaN,PNG-CB,8294-50388,NaN,NaN,NaN,NaN,NaN,NaN,85 Duke Street,WCM,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN
2,X61963,NaN,PNG-CB,8596-43320,NaN,NaN,NaN,NaN,NaN,NaN,85 Duke Street,WCM,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN
3,X61964,NaN,PNG-CB,8735-43534,NaN,NaN,NaN,NaN,NaN,NaN,85 Duke Street,WCM,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN
4,X61965,NaN,PNG-CB,8864-50705,NaN,NaN,NaN,NaN,NaN,NaN,85 Duke Street,WCM,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN


In [6]:
# Read saved data: Registers and Local DB
QLD_SRS_db = pd.read_csv("Resources/QLD_SRS.csv", encoding="utf-8")

# Display sample data
QLD_SRS_db.head()



,SRSSubstanceID,Radionuclide,Manufacturer,Model,SerialNumber,Physicalform,Chemicalform,Initialactivity,Initialactivitydate,DeviceID,DeviceType,DeviceManufacturer,DeviceModel,DeviceSerialNumber,LocationStorage,LocationUse,LocationRelocatingto
0,S48260,Am241/Be - Sealed radioactive substance,Eckert & Ziegler Isotope Products,AM1.N02,78-5887,Solid,Americium Oxide,1.48 GBQ,23-JUN-2009,D48258,Portable Soil density/moisture gauge,Troxler,3440,62988,"ROBINA, QLD 4226 - 3 Moorabbin Place",NaN,NaN
1,AT22030,Am241/Be - Sealed radioactive substance,Gammatron,AN-HP (DNS),G5224,Solid,Americium Oxide,592 GBq,12-DEC-2007,Unknown,Unknown,Unknown,Unknown,Unknown,ROMA - Store Southern Road,Throughout Queensland,NaN
2,S35491,Am241/Be - Sealed radioactive substance,Gammatron,NSR-F (AN-HP),G5380,NaN,NaN,592 GBq,28-APR-2014,Unknown,Unknown,Unknown,Unknown,Unknown,85 Duke Street ROMA QLD 4455,- Throughout Queensland,NaN
3,S35492,Am241/Be - Sealed radioactive substance,Gammatron,NSR-F (AN-HP),G5382,NaN,NaN,592 GBq,01-MAY-2014,Unknown,Unknown,Unknown,Unknown,Unknown,85 Duke Street ROMA QLD 4455,Throughout Queensland,NaN
4,S29663,Am241/Be - Sealed radioactive substance,Gammatron,NSR-F (AN-HP),G5251,Solid,Americium Oxide,592 GBq,02-MAR-2009,Unknown,Unknown,Unknown,Unknown,Unknown,85 Duke Street ROMA QLD 4455,Throughout Queensland,NaN


In [7]:
# Read saved data: Registers and Local DB
RMIS_db = pd.read_csv("Resources/RMIS_28Mar23.csv", encoding="utf-8")

# Display sample data
RMIS_db.head()



,Basin,Geounit,Country,FacilityNamewithFacilityID,BusinessLine,ModelID,SerialNo,CalibrationFrequencyinMonth,DateofLastCalibrationTest,CalibrationComplianceIndicator,...,ConfirmationInventoryFrequency,Dateoflastconfirmationinvtest,Dateofnextconfirmationinvtest,PhotoInventoryFrequency,DateofLastPhotoInvtest,DateofNextPhotoInvtest,ConfirmationPhotoComplianceIndicator,Located,PhysicalLocation,PAUControlSite
0,ASIA-ASIA,APG,Australia,Operating Base - 8509,GSS,XRD-BTX,8610/0085,NaN,(No Record),Non Compliance,...,3Month,14/03/2023,14/06/2023,12Month,9/12/2022,9/12/2023,Compliance,NaN,Shop,Jandakot Base
1,ASIA-ASIA,APG,Australia,Operating Base - 8509,GSS,XRF-SCOUT,12109147,NaN,(No Record),Non Compliance,...,3Month,9/12/2022,9/03/2023,12Month,9/12/2022,9/12/2023,Non Compliance,NaN,Tech Center,Jandakot Base
2,ASIA-ASIA,APG,Australia,Operating Base - 8509,RPE,NaN,SM|134968,12 Month,26/07/2022,Compliance,...,NaN,(Not Applicable),(Not Applicable),NaN,(Not Applicable),(Not Applicable),Ignore Case,OBS BAY,Shop,AUJAD-WLES
3,ASIA-ASIA,APG,Australia,Operating Base - 8509,RPE,NaN,SM|149029,12 Month,14/03/2022,Non Compliance,...,NaN,(Not Applicable),(Not Applicable),NaN,(Not Applicable),(Not Applicable),Ignore Case,OPCheck Bay,Shop,AUJAD-WLES
4,ASIA-ASIA,APG,Australia,Operating Base - 8509,RPE,NaN,SM|149035,12 Month,15/03/2023,Compliance,...,NaN,(Not Applicable),(Not Applicable),NaN,(Not Applicable),(Not Applicable),Ignore Case,OBS BAY,Shop,AUJAD-WLES


In [8]:
# Read saved data: Registers and Local DB
SA_db = pd.read_csv("Resources/SA.csv")

# Display sample data
SA_db.head()




,Description,Category,CertificateNumber,Isotope,Quantity,Form,Use,Manufacturer,Model,SerialNumber,ContainerModel,ContainerSerial,LocationonPremises,CurrentStatusLocation,BusinessLine,InRMISNG,Comments
0,Registration,Sealed Source,19337,Cs137,64 GBq,Sealed Solid,BORE HOLE PROBE,3M,GSR-J,3739,GCS-VC,672,"RA PIT 1, MOOMBA 5710",NaN,RPE,Yes,NaN
1,Registration,Sealed Source,20089,Am241,592 GBq,Sealed Solid,BORE HOLE PROBE,GAMMATRON,NSR-F,G5136,NCS-YC,5166,"Yellow Cell, MOOMBA 5710",NaN,RPE,Yes,NSR-FC in RMIS-NG
2,Registration,Sealed Source,22189,Cs137,62.53 GBq,Sealed Solid,BORE HOLE PROBE,AEA TECHNOLOGY,GSR-Z,A2262,NGCS-CB,11-Jun,"RA PIT, MOOMBA 5710",NaN,WCM,No,sent for RAN / PC-PC-NSSI
3,Registration,Sealed Source,22189,Am241,377.4 GBq,Sealed Solid,BORE HOLE PROBE,AEA TECHNOLOGY,NSR-U,N-392,NGCS-CB,11-Jun,"RA PIT, MOOMBA 5710",NaN,WCM,No,sent for RAN / PC-PC-NSSI
4,Registration,Sealed Source,22190,Cs137,60.59 GBq,Sealed Solid,BORE HOLE PROBE,AEA TECHNOLOGY,GSR-Z,7793B,NGCS-CB,13-May,"RA PIT, MOOMBA 5710",NaN,WCM,No,Junked - Disposed / PC-PC-NSSI


In [9]:
# Read saved data: Registers and Local DB
WA_Inventory_Amendments_db = pd.read_csv("Resources/WA_Inventory_Amendments.csv")

# Display sample data
WA_Inventory_Amendments_db.head()



,SourceN,Manufacturer,Model,SerialNumber,EquipmentUse,LocationonPremises,CurrentStatusLocation,BusinessLine,InRMISNG,Comments,Comm02,Comm03,Comm03.1
0,NaN,Schlumberger,PNG-CB,8955-51864,Logging,Jandakot Facility,NaN,WCM,Yes,Sent to Japan - RX Amendment sent 18-Oct-2022,NaN,NaN,NaN
1,22.0,Cs137,2.22 kBq,Sealed Solid,Calibration,-,EQ-602-2174,M2-136,Jandakot Facility,NaN,WCM,No,Sent to Malaysia - RS amednment sent 31-Oct-2022
2,25.0,Cs137,2.22 kBq,Sealed Solid,Calibration,-,EQ-602-2174,K1-273,Jandakot Facility,NaN,WCM,No,RS amendment sent - 7-Jun-22 (Sent to Malaysia)


In [10]:
# Read saved data: Registers and Local DB
WA_Inventory_RS_db = pd.read_csv("Resources/WA_Inventory_RS.csv")

# Display sample data
WA_Inventory_RS_db.head()



,SourceN,Isotope,Quantity,Form,PurposeorUse,Manufacturer,Model,SerialNumber,LocationonPremises,CurrentStatusLocation,BusinessLine,InRMISNG,Comments
0,1,H3,55.5 GBq,Sealed Gaseous,Gauge - logging,Schlumberger,PNG-AA,1013-51120,Balcatta Facility,NaN,RPE,Yes,NaN
1,2,Cs137,37 kBq,Sealed Solid,Calibration,Eckert & Ziegler,EQ-H343123,Q7-784,Jandakot,NaN,WCM,No,Missing in Qtrac\nInstalled in Detector# 97405...
2,3,Cs137,2.22kBq,Sealed Solid,Calibration,-,EQ-602-2174,P6-130,Jandakot,NaN,WCM,No,Amendment to add sent to WA council 22-Jul-22
3,4,Cs137,2.22kBq,Sealed Solid,Calibration,-,EQ-602-2174,P9-748,Jandakot,NaN,WCM,No,Amendment to add sent to WA council 22-Jul-22
4,5,Cs137,33.3kBq,Sealed Solid,Calibration,-,EQ-H619737,R5-597,Jandakot,NaN,WCM,Yes,Amendment to add sent to WA council 22-Jul-22


In [11]:
# Read saved data: Registers and Local DB
WA_Inventory_RX_db = pd.read_csv("Resources/WA_Inventory_RX.csv")

# Display sample data
WA_Inventory_RX_db.head()

,Model,SerialNumber,EquipmentUse,LocationonPremises,CurrentStatusLocation,BusinessLine,InRMISNG,Comments,Comments2
0,BTX,BTX-348,XRD,Jandakot Facility,NaN,WCM,No,GeoServices - registered but regulator require...,NaN
1,PNG-AA,1013-51120,Logging,Jandakot Facility,NaN,RPE,No,NaN,NaN
2,PNG-AA,1090-50652,Logging,Jandakot Facility,NaN,RPE,No,NaN,NaN
3,PNG-AA,1468-51086,Logging,Jandakot Facility,NaN,RPE,No,NaN,NaN
4,PNG-AA,1177-52252,Logging,Jandakot Facility,NaN,RPE,No,NaN,NaN


In [12]:
QLD_Reg_AR_db


,ApparatusID,ApparatusManufacturer,Model,ApparatusSerialNumber,Purpose,SubstanceID,Radionuclide,Activity,MeasurementDate,UseLocation,StorageLocation,BusinessLine,ATA,Certificate,COC,InRMIS-NG,Status,FeedbackTIM,StatusShort,Evidence
0,I-AT46800,NaN,PNG-GC,67123-50503,NaN,NaN,NaN,NaN,NaN,NaN,85 Duke Street,RPE,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN
1,X61962,NaN,PNG-CB,8294-50388,NaN,NaN,NaN,NaN,NaN,NaN,85 Duke Street,WCM,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN
2,X61963,NaN,PNG-CB,8596-43320,NaN,NaN,NaN,NaN,NaN,NaN,85 Duke Street,WCM,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN
3,X61964,NaN,PNG-CB,8735-43534,NaN,NaN,NaN,NaN,NaN,NaN,85 Duke Street,WCM,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN
4,X61965,NaN,PNG-CB,8864-50705,NaN,NaN,NaN,NaN,NaN,NaN,85 Duke Street,WCM,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,NaN,NaN,GSR-U,1006,NaN,S45289,Th,185 KBq,NaN,NaN,85 Duke Street,RPE,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN
314,NaN,NaN,GSR-U,1165,NaN,S45290,Th,185 KBq,NaN,NaN,85 Duke Street,RPE,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN
315,NaN,NaN,GSR-U,2006,NaN,S45291,Th,185 KBq,NaN,NaN,85 Duke Street,RPE,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN
316,NaN,NaN,GSR-U,2007,NaN,S45292,Th,185 KBq,NaN,NaN,85 Duke Street,RPE,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN


In [13]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/project.sqlite")

In [14]:
RMIS_db.to_sql('RMIS', con =engine)

In [15]:
QLD_IRA_db.to_sql('QLD_IRA', con = engine)

In [16]:
QLD_SRS_db.to_sql('QLD_SRS', con = engine)

In [17]:
QLD_Reg_AR_db.to_sql('QLD_Reg_AR', con = engine)

In [24]:
SA_db.to_sql('SA_Reg', con = engine)

In [21]:
WA_Inventory_RS_db.to_sql('WA_RS_Reg', con = engine)

In [22]:
WA_Inventory_RX_db.to_sql('WA_RX_Reg', con = engine)

In [25]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)
# Collect the names of tables within the database
inspector.get_table_names()


['QLD_IRA',
 'QLD_Reg_AR',
 'QLD_SRS',
 'RMIS',
 'SA_Reg',
 'WA_RS_Reg',
 'WA_RX_Reg']

In [26]:
# View all of the classes that automap found

# Using the inspector to print the column names within the 'measurement' table and its types

print ("---- QLD_IRA table information --------")

columns = inspector.get_columns('QLD_IRA')
for column in columns:
    print(column["name"], column["type"])



print ("---- QLD_Reg_AR table information --------")

columns = inspector.get_columns('QLD_Reg_AR')
for column in columns:
    print(column["name"], column["type"])


print ("\n ---- QLD_SRS table information --------")

columns = inspector.get_columns('QLD_SRS')
for column in columns:
    print(column["name"], column["type"])


print ("\n ---- RMIS table information --------")

columns = inspector.get_columns('RMIS')
for column in columns:
    print(column["name"], column["type"])


print ("\n ---- SA_Reg table information --------")

columns = inspector.get_columns('SA_Reg')
for column in columns:
    print(column["name"], column["type"])


print ("\n ---- WA_RS_Reg table information --------")

columns = inspector.get_columns('WA_RS_Reg')
for column in columns:
    print(column["name"], column["type"])


print ("\n ---- WA_RX_Reg table information --------")

columns = inspector.get_columns('WA_RX_Reg')
for column in columns:
    print(column["name"], column["type"])

---- QLD_IRA table information --------
index BIGINT
IRAApparatusID TEXT
ApparatusType TEXT
Manufacturer TEXT
Model TEXT
SerialNumber TEXT
TubeDetails TEXT
LocationStorage TEXT
LocationUse TEXT
LocationRelocatingto TEXT
SN TEXT
---- QLD_Reg_AR table information --------
index BIGINT
ApparatusID TEXT
ApparatusManufacturer FLOAT
Model TEXT
ApparatusSerialNumber TEXT
Purpose FLOAT
SubstanceID TEXT
Radionuclide TEXT
Activity TEXT
MeasurementDate FLOAT
UseLocation FLOAT
StorageLocation TEXT
BusinessLine TEXT
ATA FLOAT
Certificate FLOAT
COC FLOAT
InRMIS-NG TEXT
Status TEXT
FeedbackTIM TEXT
StatusShort TEXT
Evidence TEXT

 ---- QLD_SRS table information --------
index BIGINT
SRSSubstanceID TEXT
Radionuclide TEXT
Manufacturer TEXT
Model TEXT
SerialNumber TEXT
Physicalform TEXT
Chemicalform TEXT
Initialactivity TEXT
Initialactivitydate TEXT
DeviceID TEXT
DeviceType TEXT
DeviceManufacturer TEXT
DeviceModel TEXT
DeviceSerialNumber TEXT
LocationStorage TEXT
LocationUse TEXT
LocationRelocatingto TE

['QLD_IRA',
 'QLD_Reg_AR',
 'QLD_SRS',
 'RMIS',
 'SA_Reg',
 'WA_RS_Reg',
 'WA_RX_Reg']


In [27]:
# reflect an existing database into a new model

Base = automap_base()
Base.prepare(autoload_with=engine, reflect=True)


# reflect the tables
Base.classes.keys()

['QLD_IRA',
 'QLD_Reg_AR',
 'QLD_SRS',
 'RMIS',
 'SA_Reg',
 'WA_RS_Reg',
 'WA_RX_Reg']

In [28]:
# Save references to each table

# Map RMIS class
RMIS = Base.classes.RMIS

# Map QLD_IRA class
QLD_IRA = Base.classes.QLD_IRA

# Map QLD_Reg_AR class
QLD_Reg_AR = Base.classes.QLD_Reg_AR

# Map QLD_SRS class
QLD_SRS_AR = Base.classes.QLD_SRS

# Map SA_Reg class
SA_Reg = Base.classes.SA_Reg

# Map WA_RS_Reg class
WA_RS_Reg = Base.classes.WA_RS_Reg

# Map WA_RX_Reg class
WA_RX_Reg = Base.classes.WA_RX_Reg

In [29]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [30]:

print ("\n ---- RMIS table information --------")

columns = inspector.get_columns('RMIS')
for column in columns:
    print(column["name"], column["type"])


 ---- RMIS table information --------
index BIGINT
Basin TEXT
Geounit TEXT
Country TEXT
FacilityNamewithFacilityID TEXT
BusinessLine TEXT
ModelID TEXT
SerialNo TEXT
CalibrationFrequencyinMonth TEXT
DateofLastCalibrationTest TEXT
CalibrationComplianceIndicator TEXT
LeakTestFrequencyinMonth TEXT
DateofLastLeakTest TEXT
LeakTestComplianceIndicator TEXT
LastLeakTestResultRecorded TEXT
LeakTestResultComplianceIndicator TEXT
ConfirmationInventoryFrequency TEXT
Dateoflastconfirmationinvtest TEXT
Dateofnextconfirmationinvtest TEXT
PhotoInventoryFrequency TEXT
DateofLastPhotoInvtest TEXT
DateofNextPhotoInvtest TEXT
ConfirmationPhotoComplianceIndicator TEXT
Located TEXT
PhysicalLocation TEXT
PAUControlSite TEXT


In [31]:
# Photo inventory dates dates


print ("\n----- Total dates------\n")
totald = session.query(func.count(RMIS.DateofLastPhotoInvtest)).\
    filter(RMIS.PhotoInventoryFrequency == '12Month').\
    order_by(RMIS.DateofLastPhotoInvtest).all()
print (totald)

# Earliest Date

print ("\n----- Earliest Date------\n")
date = session.query(RMIS.DateofLastPhotoInvtest).order_by(RMIS.DateofLastPhotoInvtest).\
    filter(RMIS.PhotoInventoryFrequency == '12Month').\
    order_by(RMIS.DateofLastPhotoInvtest).first()
print (date)


# Latest Date

print ("\n----- Latest Date-------\n")
date = session.query(RMIS.DateofLastPhotoInvtest).order_by(RMIS.DateofLastPhotoInvtest.desc()).\
    filter(RMIS.PhotoInventoryFrequency == '12Month').\
    order_by(RMIS.DateofLastPhotoInvtest).first()
print (date)


----- Total dates------

[(833,)]

----- Earliest Date------

('1/02/2023',)

----- Latest Date-------

('9/12/2022',)


In [32]:
print ("\n----- Records in a location------\n")
totald = session.query(RMIS.BusinessLine,func.count(RMIS.FacilityNamewithFacilityID)).\
    filter(RMIS.FacilityNamewithFacilityID == 'Jandakot Base OFS Operating Base - 8509').\
    group_by(RMIS.BusinessLine).\
    all()
print (totald)


----- Records in a location------

[]


In [33]:
print ("\n----- Records in a location------\n")
totald = session.query(RMIS.FacilityNamewithFacilityID, RMIS.BusinessLine,func.count(RMIS.FacilityNamewithFacilityID)).\
     group_by(RMIS.FacilityNamewithFacilityID).\
     order_by(RMIS.BusinessLine).\
    all()
print (totald)


----- Records in a location------

[('Base - 7559', 'GSS', 254), ('Operating Base - 8509', 'GSS', 138), ('Base - 1205', 'RPE', 6), ('Base - 1282', 'RPE', 35), ('Base - 13061', 'RPE', 88), ('Base - 1548', 'RPE', 9), ('Base - 1603', 'RPE', 20), ('Base - 1644', 'RPE', 32), ('Base - 1696', 'RPE', 24), ('Base - 1839', 'RPE', 63), ('Base - 1943', 'RPE', 13), ('Base - 2074', 'RPE', 52), ('Base - 2171', 'RPE', 45), ('Base - 255', 'RPE', 213), ('Base - 560', 'RPE', 61), ('Base - 789', 'RPE', 76), ('Base - 8167', 'RPE', 16), ('Yard Cilandak - 2763', 'RPE', 4), ('Base - 10493', 'RPS', 7), ('Base - 2073', 'WCM', 5), ('Warehouse - 18099', 'WCM', 99), ('Center - 2353', 'WPRS', 5)]


In [34]:
# Calibration compliance

print ("\n----- Calibration compliance------\n")
CalComp = session.query(RMIS.FacilityNamewithFacilityID, RMIS.BusinessLine, RMIS.CalibrationComplianceIndicator, func.count(RMIS.CalibrationComplianceIndicator)).\
    group_by(RMIS.FacilityNamewithFacilityID).\
    group_by(RMIS.BusinessLine).\
    group_by(RMIS.CalibrationComplianceIndicator).\
    all()
print (CalComp)


----- Calibration compliance------

[('Base - 10493', 'RPS', 'Compliance', 3), ('Base - 10493', 'RPS', 'Non Compliance', 4), ('Base - 1205', 'RPE', 'Non Compliance', 6), ('Base - 1282', 'RPE', 'Compliance', 2), ('Base - 1282', 'RPE', 'Non Compliance', 31), ('Base - 1282', 'WCM', 'Compliance', 1), ('Base - 1282', 'WCM', 'Non Compliance', 1), ('Base - 13061', 'RPE', 'Compliance', 4), ('Base - 13061', 'RPE', 'Non Compliance', 20), ('Base - 13061', 'WCM', 'Action Needed', 2), ('Base - 13061', 'WCM', 'Compliance', 16), ('Base - 13061', 'WCM', 'Non Compliance', 46), ('Base - 1548', 'RPE', 'Compliance', 1), ('Base - 1548', 'RPE', 'Critical', 1), ('Base - 1548', 'RPE', 'Non Compliance', 7), ('Base - 1603', 'RPE', 'Non Compliance', 20), ('Base - 1644', 'RPE', 'Action Needed', 2), ('Base - 1644', 'RPE', 'Compliance', 2), ('Base - 1644', 'RPE', 'Non Compliance', 27), ('Base - 1644', 'WCM', 'Non Compliance', 1), ('Base - 1696', 'RPE', 'Compliance', 3), ('Base - 1696', 'RPE', 'Non Compliance', 19)

In [35]:
CalComp


[('Base - 10493', 'RPS', 'Compliance', 3),
 ('Base - 10493', 'RPS', 'Non Compliance', 4),
 ('Base - 1205', 'RPE', 'Non Compliance', 6),
 ('Base - 1282', 'RPE', 'Compliance', 2),
 ('Base - 1282', 'RPE', 'Non Compliance', 31),
 ('Base - 1282', 'WCM', 'Compliance', 1),
 ('Base - 1282', 'WCM', 'Non Compliance', 1),
 ('Base - 13061', 'RPE', 'Compliance', 4),
 ('Base - 13061', 'RPE', 'Non Compliance', 20),
 ('Base - 13061', 'WCM', 'Action Needed', 2),
 ('Base - 13061', 'WCM', 'Compliance', 16),
 ('Base - 13061', 'WCM', 'Non Compliance', 46),
 ('Base - 1548', 'RPE', 'Compliance', 1),
 ('Base - 1548', 'RPE', 'Critical', 1),
 ('Base - 1548', 'RPE', 'Non Compliance', 7),
 ('Base - 1603', 'RPE', 'Non Compliance', 20),
 ('Base - 1644', 'RPE', 'Action Needed', 2),
 ('Base - 1644', 'RPE', 'Compliance', 2),
 ('Base - 1644', 'RPE', 'Non Compliance', 27),
 ('Base - 1644', 'WCM', 'Non Compliance', 1),
 ('Base - 1696', 'RPE', 'Compliance', 3),
 ('Base - 1696', 'RPE', 'Non Compliance', 19),
 ('Base - 1696

In [36]:
#Leak Test compliance

print ("\n----- Leak Testcompliance------\n")
LeakTestComp = session.query(RMIS.FacilityNamewithFacilityID, RMIS.BusinessLine, RMIS.LeakTestComplianceIndicator, func.count(RMIS.LeakTestComplianceIndicator)).\
    group_by(RMIS.FacilityNamewithFacilityID).\
    group_by(RMIS.BusinessLine).\
    group_by(RMIS.LeakTestComplianceIndicator).\
    all()
print (LeakTestComp)


----- Leak Testcompliance------

[('Base - 10493', 'RPS', 'Compliance', 4), ('Base - 10493', 'RPS', 'Non Compliance', 3), ('Base - 1205', 'RPE', 'Compliance', 5), ('Base - 1205', 'RPE', 'Not Applicable', 1), ('Base - 1282', 'RPE', 'Compliance', 11), ('Base - 1282', 'RPE', 'Non Compliance', 2), ('Base - 1282', 'RPE', 'Not Applicable', 20), ('Base - 1282', 'WCM', 'Non Compliance', 1), ('Base - 1282', 'WCM', 'Not Applicable', 1), ('Base - 13061', 'RPE', 'Compliance', 4), ('Base - 13061', 'RPE', 'Critical', 3), ('Base - 13061', 'RPE', 'Non Compliance', 4), ('Base - 13061', 'RPE', 'Not Applicable', 13), ('Base - 13061', 'WCM', 'Compliance', 8), ('Base - 13061', 'WCM', 'Non Compliance', 23), ('Base - 13061', 'WCM', 'Not Applicable', 33), ('Base - 1548', 'RPE', 'Compliance', 4), ('Base - 1548', 'RPE', 'Non Compliance', 2), ('Base - 1548', 'RPE', 'Not Applicable', 3), ('Base - 1603', 'RPE', 'Compliance', 11), ('Base - 1603', 'RPE', 'Not Applicable', 9), ('Base - 1644', 'RPE', 'Compliance', 10

In [37]:
# Photo compliance

print ("\n----- Photo compliance------\n")
PhotoComp = session.query(RMIS.FacilityNamewithFacilityID, RMIS.BusinessLine, RMIS.ConfirmationPhotoComplianceIndicator, func.count(RMIS.ConfirmationPhotoComplianceIndicator)).\
    group_by(RMIS.FacilityNamewithFacilityID).\
    group_by(RMIS.BusinessLine).\
    group_by(RMIS.ConfirmationPhotoComplianceIndicator).\
    all()
print (PhotoComp)


----- Photo compliance------

[('Base - 10493', 'RPS', 'Compliance', 4), ('Base - 10493', 'RPS', 'Ignore Case', 3), ('Base - 1205', 'RPE', 'Action Needed', 1), ('Base - 1205', 'RPE', 'Compliance', 2), ('Base - 1205', 'RPE', 'Non Compliance', 3), ('Base - 1282', 'RPE', 'Compliance', 29), ('Base - 1282', 'RPE', 'Ignore Case', 2), ('Base - 1282', 'RPE', 'Not Applicable', 2), ('Base - 1282', 'WCM', 'Ignore Case', 1), ('Base - 1282', 'WCM', 'Non Compliance', 1), ('Base - 13061', 'RPE', 'Compliance', 9), ('Base - 13061', 'RPE', 'Critical', 3), ('Base - 13061', 'RPE', 'Ignore Case', 4), ('Base - 13061', 'RPE', 'Non Compliance', 3), ('Base - 13061', 'RPE', 'Not Applicable', 5), ('Base - 13061', 'WCM', 'Action Needed', 3), ('Base - 13061', 'WCM', 'Compliance', 23), ('Base - 13061', 'WCM', 'Critical', 10), ('Base - 13061', 'WCM', 'Ignore Case', 19), ('Base - 13061', 'WCM', 'Non Compliance', 4), ('Base - 13061', 'WCM', 'Not Applicable', 5), ('Base - 1548', 'RPE', 'Compliance', 5), ('Base - 1548'

In [38]:
# Total dates

print ("\n----- Total dates------\n")
totald = session.query(func.count(RMIS.Geounit)).all()
print (totald)


----- Total dates------

[(1265,)]


In [39]:
print ("\n----- Photo compliance------\n")
PhotoComp = session.query(RMIS.Geounit, RMIS.Country, RMIS.FacilityNamewithFacilityID, RMIS.BusinessLine, RMIS.ConfirmationPhotoComplianceIndicator, func.count(RMIS.ConfirmationPhotoComplianceIndicator)).\
    group_by(RMIS.FacilityNamewithFacilityID).\
    group_by(RMIS.BusinessLine).\
    group_by(RMIS.ConfirmationPhotoComplianceIndicator).\
    all()
print (PhotoComp)



----- Photo compliance------

[('APG', 'Australia', 'Base - 10493', 'RPS', 'Compliance', 4), ('APG', 'Australia', 'Base - 10493', 'RPS', 'Ignore Case', 3), ('ING', 'Indonesia', 'Base - 1205', 'RPE', 'Action Needed', 1), ('ING', 'Indonesia', 'Base - 1205', 'RPE', 'Compliance', 2), ('ING', 'Indonesia', 'Base - 1205', 'RPE', 'Non Compliance', 3), ('CHG', 'China', 'Base - 1282', 'RPE', 'Compliance', 29), ('CHG', 'China', 'Base - 1282', 'RPE', 'Ignore Case', 2), ('CHG', 'China', 'Base - 1282', 'RPE', 'Not Applicable', 2), ('CHG', 'China', 'Base - 1282', 'WCM', 'Ignore Case', 1), ('CHG', 'China', 'Base - 1282', 'WCM', 'Non Compliance', 1), ('CHG', 'China', 'Base - 13061', 'RPE', 'Compliance', 9), ('CHG', 'China', 'Base - 13061', 'RPE', 'Critical', 3), ('CHG', 'China', 'Base - 13061', 'RPE', 'Ignore Case', 4), ('CHG', 'China', 'Base - 13061', 'RPE', 'Non Compliance', 3), ('CHG', 'China', 'Base - 13061', 'RPE', 'Not Applicable', 5), ('CHG', 'China', 'Base - 13061', 'WCM', 'Action Needed', 3),

In [43]:
PhotoComp


[('APG', 'Australia', 'Base - 10493', 'RPS', 'Compliance', 4),
 ('APG', 'Australia', 'Base - 10493', 'RPS', 'Ignore Case', 3),
 ('ING', 'Indonesia', 'Base - 1205', 'RPE', 'Action Needed', 1),
 ('ING', 'Indonesia', 'Base - 1205', 'RPE', 'Compliance', 2),
 ('ING', 'Indonesia', 'Base - 1205', 'RPE', 'Non Compliance', 3),
 ('CHG', 'China', 'Base - 1282', 'RPE', 'Compliance', 29),
 ('CHG', 'China', 'Base - 1282', 'RPE', 'Ignore Case', 2),
 ('CHG', 'China', 'Base - 1282', 'RPE', 'Not Applicable', 2),
 ('CHG', 'China', 'Base - 1282', 'WCM', 'Ignore Case', 1),
 ('CHG', 'China', 'Base - 1282', 'WCM', 'Non Compliance', 1),
 ('CHG', 'China', 'Base - 13061', 'RPE', 'Compliance', 9),
 ('CHG', 'China', 'Base - 13061', 'RPE', 'Critical', 3),
 ('CHG', 'China', 'Base - 13061', 'RPE', 'Ignore Case', 4),
 ('CHG', 'China', 'Base - 13061', 'RPE', 'Non Compliance', 3),
 ('CHG', 'China', 'Base - 13061', 'RPE', 'Not Applicable', 5),
 ('CHG', 'China', 'Base - 13061', 'WCM', 'Action Needed', 3),
 ('CHG', 'China

In [44]:
PhotoComp_df = pd.DataFrame(PhotoComp, columns=['FacilityNamewithFacilityID', 'BusinessLine', 'ConfirmationPhotoComplianceIndicator','Count'])

ValueError: 4 columns passed, passed data had 6 columns

In [42]:
PhotoComp_df

NameError: name 'PhotoComp_df' is not defined

In [52]:
 PhotoComp_data_out = []

In [56]:
len(PhotoComp)

130

In [45]:
PhotoComp[0].FacilityNamewithFacilityID

'Base - 10493'

In [46]:
PhotoComp[0].BusinessLine

'RPS'

In [47]:
PhotoComp[0].ConfirmationPhotoComplianceIndicator

'Compliance'

In [48]:
PhotoComp[0][3]

'RPS'

In [49]:
    for row in PhotoComp:
        PhotoComp_dict = {}
        PhotoComp_dict["FacilityNamewithFacilityID"]=row.FacilityNamewithFacilityID
        PhotoComp_dict["BusinessLine"]=row.BusinessLine
        PhotoComp_dict["ConfirmationPhotoComplianceIndicator"]=row.ConfirmationPhotoComplianceIndicator
        PhotoComp_dict["Count"]=row[3]

        PhotoComp_data_out.append(PhotoComp_dict)

NameError: name 'PhotoComp_data_out' is not defined

In [50]:
PhotoComp_data_out

NameError: name 'PhotoComp_data_out' is not defined

In [43]:
# Close Session
session.close()

In [51]:
PhotoComp

[('APG', 'Australia', 'Base - 10493', 'RPS', 'Compliance', 4),
 ('APG', 'Australia', 'Base - 10493', 'RPS', 'Ignore Case', 3),
 ('ING', 'Indonesia', 'Base - 1205', 'RPE', 'Action Needed', 1),
 ('ING', 'Indonesia', 'Base - 1205', 'RPE', 'Compliance', 2),
 ('ING', 'Indonesia', 'Base - 1205', 'RPE', 'Non Compliance', 3),
 ('CHG', 'China', 'Base - 1282', 'RPE', 'Compliance', 29),
 ('CHG', 'China', 'Base - 1282', 'RPE', 'Ignore Case', 2),
 ('CHG', 'China', 'Base - 1282', 'RPE', 'Not Applicable', 2),
 ('CHG', 'China', 'Base - 1282', 'WCM', 'Ignore Case', 1),
 ('CHG', 'China', 'Base - 1282', 'WCM', 'Non Compliance', 1),
 ('CHG', 'China', 'Base - 13061', 'RPE', 'Compliance', 9),
 ('CHG', 'China', 'Base - 13061', 'RPE', 'Critical', 3),
 ('CHG', 'China', 'Base - 13061', 'RPE', 'Ignore Case', 4),
 ('CHG', 'China', 'Base - 13061', 'RPE', 'Non Compliance', 3),
 ('CHG', 'China', 'Base - 13061', 'RPE', 'Not Applicable', 5),
 ('CHG', 'China', 'Base - 13061', 'WCM', 'Action Needed', 3),
 ('CHG', 'China

In [52]:
PhotoComp_df = pd.DataFrame(PhotoComp, columns=['Geounit', 'Country', 'FacilityNamewithFacilityID', 'BusinessLine', 'ConfirmationPhotoComplianceIndicator','Count'])

In [53]:
bases= PhotoComp_df['FacilityNamewithFacilityID'].unique()

In [54]:
bases

array(['Base - 10493', 'Base - 1205', 'Base - 1282', 'Base - 13061',
       'Base - 1548', 'Base - 1603', 'Base - 1644', 'Base - 1696',
       'Base - 1839', 'Base - 1943', 'Base - 2073', 'Base - 2074',
       'Base - 2171', 'Base - 255', 'Base - 560', 'Base - 7559',
       'Base - 789', 'Base - 8167', 'Center - 2353',
       'Operating Base - 8509', 'Warehouse - 18099',
       'Yard Cilandak - 2763'], dtype=object)

In [55]:
countries = PhotoComp_df['Country'].unique()

In [89]:
countries

array(['China', 'Indonesia', 'Australia', 'Japan', 'New Zealand',
       'Papua New Guinea', 'Taiwan, Province Of China'], dtype=object)

In [56]:
cities = ['China Chengdu OFS Base - 560', 'China Huizhou Radioactive Warehouse - 18099', 'Indonesia JOP - Cikarang Integrated Base CIB) OFS Base - 7559', 'Indonesia JOP - Elnusa Open Yard Cilandak - 2763','Australia Jandakot Base OFS Operating Base - 8509','Indonesia KAL - Manggar OFS Base - 255', 'China Korla Integrated Base - 1282', 'China LinGang Integrated Base - 13061', 'Australia Moomba WL Operating Base - 1603', 'Japan Nagaoka OFS Base - 1644', 'New Zealand New Plymouth OFS Base Integrated Base - 1696', 'Australia Newburn Base - 1839', 'Papua New Guinea', 'PNG OFS Base Integrated Base - 8167', 'Australia Roma Smith Pathfinder Operating Base - 2073', 'Australia Roma WL Operating Base - 2074', 'Indonesia SUM - Duri OFS Base - 789',  'Indonesia SUM - Jambi WL Base - 1205', 'Indonesia SUM - Prabumulih OFS Base - 1943', 'China Shekou Oilfield Services Base - 2171', 'Taiwan, Province Of China Taiwan Miaoli base - 1548', 'Australia Toowoomba OFS Operating Base - 10493','China XiQing Product Center - 2353']

In [ ]:
# Close Session
session.close()